Continues from A006h

In [3]:
import os
import sys
import warnings
import random
import copy
import pickle
import glob

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

sys.path.append('../common')
import data_io_utils
import paths
import constants
import utils

sys.path.append('../A003_policy_optimization/')
import models
import A003_common

import A006_common
from unirep import babbler1900 as babbler
import sequence_selection


%reload_ext autoreload
%autoreload 2

## Sync data.

We only need to sync the selected sequence folders.

In [ ]:
!aws s3 cp s3://low-n-protein-engineering-public/chip_1/simulated_annealing/CBDAS/ ../../data/s3/chip_1/simulated_annealing/CBDAS/ --exclude "*" --include "*selected_seqs*" --recursive

In [ ]:
!aws s3 cp s3://low-n-protein-engineering-public/chip_1/simulated_annealing/THCAS/ ../../data/s3/chip_1/simulated_annealing/THCAS/ --exclude "*" --include "*selected_seqs*" --recursive

In [5]:
CBDAS_selected_seqs_dir = os.path.join(data_io_utils.S3_DATA_ROOT, 'chip_1', 'simulated_annealing', 'CBDAS')
THCAS_selected_seqs_dir = os.path.join(data_io_utils.S3_DATA_ROOT, 'chip_1', 'simulated_annealing', 'THCAS')

## Load the sequence allocation table. 

This is copied and pasted from here ("Tabular Seq Allocation" tab): https://docs.google.com/spreadsheets/d/1UVfD0TYKvWi2CPM9tec1qtry053UarqxRvx8BKR7Ytw/edit#gid=533377689

In [6]:
seq_alloc_df = pd.read_csv('Chip_1_Sequence_Allocation.txt', sep='\t')
seq_alloc_df.head()

protein        base_model n_train trust_radius seed  seqs_allocated
0     GFP  ET_Global_Init_1      24           15    0             300
1     GFP  ET_Global_Init_1      24           15    1             300
2     GFP  ET_Global_Init_1      24           15    2             300
3     GFP  ET_Global_Init_1      24           15    3             300
4     GFP  ET_Global_Init_1      24           15    4             300

In [7]:
selected_seq_dfs = []
for i,r in seq_alloc_df.iterrows():
    if r['n_train'] == 'LARGE_N ??':
        continue # Never did these experiments.
        
    if r['n_train'] == 'Grigory\'s Special Set':
        gk_df = pd.read_csv('GK_Special_Set_20190812_df_version2_cbdas_greedy.csv')
        gk_df['protein'] = 'CBDAS'
        selected_seq_dfs.append(gk_df)
        
        continue # for now
    
    protein = r['protein']
    glob_exp = '*' + r['base_model'] + '-00' + str(r['n_train']) + '-0' + r['seed'] + '-*-selected_seqs'
    
    # Grab the directory containing the sequences selected from simulated annealing (see A006h)
    if protein == 'CBDAS':
        selected_seqs_dir = glob.glob(os.path.join(CBDAS_selected_seqs_dir, glob_exp))
                
        if int(r['trust_radius']) == 15:
            selected_seqs_dir = [s for s in selected_seqs_dir if ('SparseRefit' not in s) and ('SmallTrust' not in s)]
        elif int(r['trust_radius']) == 7:
            selected_seqs_dir = [s for s in selected_seqs_dir if ('SparseRefit' not in s) and ('SmallTrust' in s)]
        else:
            assert False, 'unexpected trust radius'
                        
        assert len(selected_seqs_dir) == 1
        
        selected_seqs_dir = selected_seqs_dir[0]
        
    elif protein == 'THCAS':
        
        selected_seqs_dir = glob.glob(os.path.join(THCAS_selected_seqs_dir, glob_exp))
                
        if int(r['trust_radius']) == 15:
            selected_seqs_dir = [s for s in selected_seqs_dir if ('SparseRefit' not in s) and ('LargeMut' in s)]
        elif int(r['trust_radius']) == 7:
            selected_seqs_dir = [s for s in selected_seqs_dir if ('SparseRefit' not in s) and ('LargeMut' not in s)]
        else:
            assert False, 'unexpected trust radius'
            
        assert len(selected_seqs_dir) == 1
        
        selected_seqs_dir = selected_seqs_dir[0]
        
    print(selected_seqs_dir)
    seq_file = os.path.join(selected_seqs_dir, 'selected_seqs_df.pkl')
    seq_df = pd.read_pickle(seq_file)
    seq_df['protein'] = r['protein']
    
    selected_seq_dfs.append(seq_df.iloc[:r['seqs_allocated']])    

aggregated_selected_seq_df = pd.concat(selected_seq_dfs, axis=0, sort=False)        

NameError: name 'selected_seqs_dir' is not defined

In [8]:
print(aggregated_selected_seq_df.shape)
aggregated_selected_seq_df.head()

NameError: name 'aggregated_selected_seq_df' is not defined

In [9]:
aggregated_selected_seq_df.tail()

NameError: name 'aggregated_selected_seq_df' is not defined

## Export

In [9]:
output_file = os.path.join(data_io_utils.S3_DATA_ROOT, 'chip_1', 
        'A006i_aggregated_selected_seqs_for_cloning_validation.pkl')
aggregated_selected_seq_df.to_pickle(output_file)

# Post publication: Commenting out original sync code as it will not work to the read-only bucket.
#data_io_utils.sync_local_path_to_s3(output_file, is_single_file=True)